## Libraries used
This demonstration uses scipy for its non-linear solver.  For machine learning components, it uses torch, gymnasium, and stable-baselines3.  Additional libraries may be imported to assist with evaluation or presentation of results.

In [ ]:
import numpy as np
from scipy.optimize import minimize

In [ ]:
import gymnasium as gym
from gymnasium import spaces

In [ ]:
import torch as th

In [ ]:
from stable_baselines3.ppo.policies import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback

## Data needed
In practice, one would likely use historical market data to build models.  This poses many challenges, as large and robust datasets are often currated by vendors.  Some free options are available, especially outside of commercial use.  However, to simplify things for this demonstration, we will simulate market data.

Note that for machine learning applications using historical market data, there are often complexities related to how data is sampled.  If one is not careful, it is easy to overfit to limited historical data.

In [ ]:
def generate_market_params(assets, N, seed=None):
    if not seed is None:
        np.random.seed(seed)

    uncorrelated_returns = np.random.normal(0.01,0.02,size=(N,assets))
    random_weights = np.random.normal(0,.3,size=(assets,assets))
    correlated_returns = uncorrelated_returns@random_weights
    mean_return = correlated_returns.mean(0)
    cov_return = np.cov(correlated_returns,rowvar=False)
    return mean_return, cov_return

def generate_market_returns(mean,cov,N,seed=None):
    if not seed is None:
        np.random.seed(seed)

    correlated_returns = np.random.multivariate_normal(mean,cov,N)
    return correlated_returns

In [ ]:
mu, Sig = generate_market_params(5,100)
simulated_returns = generate_market_returns(mu,Sig,100)

## Traditional portfolio optimization
Traditionally, one would construct a portfolio by first defining the distribution of returns.  This generally requires an assumption for the distribution.  For simplicity, we will assume the assets follow a multivariate normal distribution.  Much research shows that fatter tails are common, and for some assets, a normal distribution wouldn't make sense - but thsi is just for exposition.  A multivariate normal distribution is described by two parameters, $\mu$ and $\Sigma$.  $\mu$ is a vector of means for each of the random normals, and $\Sigma$ is a matrix of their covariances.

Once these parameters are set, then we can calculate various metrics, such as the expected return, the expected standard deviation, the sharpe ratio, the value-at-risk, etc.  We can use these metrics along with a solver, such as the solver in scipy, to minimize or maximize the metric's value over all feasible allocations.

In [ ]:
mu_est = simulated_returns.mean(0)
cov_est = np.cov(simulated_returns,rowvar=False)

def expected_sharpe_ratio(x,mu,sigma,risk_free_rate=0):
    exp_return = x@mu
    std_dev = np.sqrt(x@sigma@x)
    return -(exp_return-risk_free_rate)/std_dev

def canonical_optim(mu,sigma,risk_free_rate=0):
    constraints=[{'type':'eq','fun':lambda x: x.sum(0)-1}]
    rslt = minimize(fun=expected_sharpe_ratio
                   ,x0=[1,0,0,0,0]
                   ,constraints=constraints
                   ,args=(mu,sigma, risk_free_rate)
                   ,bounds=((0,1),(0,1),(0,1),(0,1),(0,1)))
    return rslt.x.round(2)

In [ ]:
canonical_optim(mu_est,cov_est)

array([0.  , 0.07, 0.  , 0.93, 0.  ])

## Skipping estimation of parameters
Instead of explicitly estimating the parameters of the distribution for the asset returns, we can calculate historical metrics directly.  This can help us avoid some of the pitfalls of bad assumptions for the distribution, but it doesn't allow us to add in expert judgement about the value of the parameters (e.g. if we think a certain asset classes will have higher performance in the future).

In [ ]:
def historical_sharpe_ratio(x,returns,risk_free_rate=0):
    port_return = returns@x
    port_mean = port_return.mean()
    port_std_dev = port_return.std()
    return -(port_mean-risk_free_rate)/port_std_dev

def data_optim(returns,risk_free_rate=0):
    constraints=[{'type':'eq','fun':lambda x: x.sum(0)-1}]
    rslt = minimize(fun=historical_sharpe_ratio
                   ,x0=[1,0,0,0,0]
                   ,constraints=constraints
                   ,args=(returns, risk_free_rate)
                   ,bounds=((0,1),(0,1),(0,1),(0,1),(0,1)))
    return rslt.x.round(2)

In [ ]:
data_optim(simulated_returns)

array([0.  , 0.07, 0.  , 0.93, 0.  ])

## Very simple application of reinforcement learning
Another approach to finding an optimal asset allocation is to use reinforcement learning.

To start, let's think about the problem the same way we thought about it in the traditional setting - we want to maximize the historical sharpe ratio by selecting the best asset allocation.  To do this, we set the reward as the sharpe ratio and try and find the policy that will maximize it.

In [ ]:
class InvestmentGame0(gym.Env):
    def __init__(self,returns,risk_free_rate=0):
        super(InvestmentGame0,self).__init__()
        self.action_space = spaces.Box(
            low=0,high=1,shape=(5,),dtype=np.float32)
        self.observation_space=spaces.Box(
            low=-1, high=1, shape=(returns.shape),dtype=np.float32)
        self.reward_range=(-10,10)
        self.returns = returns
        self.risk_free_rate = risk_free_rate

    def step(self,action):
        self.current_action = action
        if action.sum()==0:
            self.weights = (action+1)/5
        else:
            self.weights = action/action.sum()

        port_returns = self.returns@self.weights
        sharpe = (port_returns.mean()-self.risk_free_rate)/port_returns.std()
        reward = sharpe.clip(-10,10)
        terminated = 1
        obs =self._get_obs()
        info = {"data":1}
        return obs,reward,terminated,0,info
    def reset(self,seed=None):
        if not seed is None:
            np.random.seed(seed)
        obs = self._get_obs()
        return obs, {'data':1}
    def _get_obs(self):
        obs = self.returns
        return obs

In [ ]:
env = DummyVecEnv([lambda: InvestmentGame0(simulated_returns)])
policy_kwargs = dict(activation_fn=th.nn.ReLU,
                    net_arch=[dict(pi=[128,64,32],vf=[128,64,32])])
model0 = PPO(MlpPolicy, env, learning_rate=0.02, batch_size=1000, gamma=1, n_steps=10000, policy_kwargs=policy_kwargs, verbose=1)
obs = env.reset()
model0.learn(total_timesteps=300000)

Using cpu device


C:\Users\tlhee\AppData\Local\Continuum\anaconda3\envs\port_mgmt\Lib\site-packages\stable_baselines3\common\policies.py:484: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


------------------------------
| time/              |       |
|    fps             | 1190  |
|    iterations      | 1     |
|    time_elapsed    | 8     |
|    total_timesteps | 10000 |
------------------------------
---------------------------------------
| time/                   |           |
|    fps                  | 1117      |
|    iterations           | 2         |
|    time_elapsed         | 17        |
|    total_timesteps      | 20000     |
| train/                  |           |
|    approx_kl            | 0.8414378 |
|    clip_fraction        | 0.843     |
|    clip_range           | 0.2       |
|    entropy_loss         | -6.79     |
|    explained_variance   | 0         |
|    learning_rate        | 0.02      |
|    loss                 | -0.106    |
|    n_updates            | 10        |
|    policy_gradient_loss | -0.142    |
|    std                  | 0.923     |
|    value_loss           | 0.089     |
---------------------------------------
-----------------------

-----------------------------------------
| time/                   |             |
|    fps                  | 935         |
|    iterations           | 13          |
|    time_elapsed         | 139         |
|    total_timesteps      | 130000      |
| train/                  |             |
|    approx_kl            | 0.023140555 |
|    clip_fraction        | 0.367       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.45       |
|    explained_variance   | 0           |
|    learning_rate        | 0.02        |
|    loss                 | -0.0291     |
|    n_updates            | 120         |
|    policy_gradient_loss | -0.0258     |
|    std                  | 0.399       |
|    value_loss           | 0.000854    |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 930         |
|    iterations           | 14          |
|    time_elapsed         | 150   

----------------------------------------
| time/                   |            |
|    fps                  | 912        |
|    iterations           | 24         |
|    time_elapsed         | 263        |
|    total_timesteps      | 240000     |
| train/                  |            |
|    approx_kl            | 0.08060528 |
|    clip_fraction        | 0.601      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.471     |
|    explained_variance   | 0          |
|    learning_rate        | 0.02       |
|    loss                 | 0.00144    |
|    n_updates            | 230        |
|    policy_gradient_loss | 0.0149     |
|    std                  | 0.269      |
|    value_loss           | 1.17e-06   |
----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 912        |
|    iterations           | 25         |
|    time_elapsed         | 274        |
|    total_times

### Evaluating results
In addition to the many typical metrics provided for RL models, which are printed by stable-baselines3, we may want to evaluate our model based on the specific circumstances and objectives of our problem.

As can be seen, below, the policy model is able to learn to allocate funds very similarly to the traditional optimization model.  This is encouraging!  It means that our environment and training alogrithm are not completely useless...

In [ ]:
def alloc_predict(returns,model):
    raw_alloc = model.predict(returns)[0]
    if raw_alloc.sum() == 0:
        alloc = (raw_alloc+1)/5
    else:
        alloc = raw_alloc/raw_alloc.sum()
    return alloc

alloc_predict(simulated_returns,model0)

array([0., 0., 0., 1., 0.], dtype=float32)

In [ ]:
data_optim(simulated_returns)

array([0.  , 0.07, 0.  , 0.93, 0.  ])

#### A different set of market data
Unfortunately, the policy is not able to find an optimal strategy given different market data.  It only saw a single set of data in its training, so this should be expected.

In [ ]:
mu2,Sig2 = generate_market_params(5,100)
simulated_returns2 = generate_market_returns(mu2,Sig2,100)
alloc_predict(simulated_returns2,model0)

array([0., 0., 0., 1., 0.], dtype=float32)

In [ ]:
data_optim(simulated_returns2)

array([0.  , 0.  , 0.  , 0.29, 0.71])

### Another strategy for testing our model
In this case, we can look across many different markets (i.e. simulate many different historical market datasets) and see how similar our policy models is to the traditional model.  In this case, a plausible benchmark is to just compare our model to an 'even split' across each asset (i.e. 20% allocation).  If our policy isn't better than the benchmark, we probably should be concerned - and this is obviously the case.  Of course, this relates to the limited data seen in training.

In [ ]:
def test_model_variance(model,N=1000):
    variance = []
    benchmark = []
    for i in range(N):
        mu,Sig = generate_market_params(5,100)
        simulated_returns = generate_market_returns(mu,Sig,100)
        predict_alloc = alloc_predict(simulated_returns,model)
        optimal_alloc = data_optim(simulated_returns)
        variance.append(((predict_alloc-optimal_alloc)**2).sum())
        benchmark.append(((0.2-optimal_alloc)**2).sum())
    return variance,benchmark

model_sq_dif, bchmk_sq_dif = test_model_variance(model0)

np.mean(model_sq_dif), np.mean(bchmk_sq_dif)

(1.1853248797530855, 0.4009954000000001)

### Looking at the rewards
A final way to evaluate the model is to compare the reward against a benchmark - in our case, the benchmark is the traditional approach.  Of course, the results are quite similar - but, agian, our environment only has a single set of historical returns so this result is not terribly interesting.

In [ ]:
def policy_tester(environment,policy_model,N=1000,simulated_returns=None):
    if simulated_returns is None:
        env = environment()
    else:
        env = environment(simulated_returns)
    obs,_ = env.reset()
    total_reward = 0
    for i in range(N):
        obs,reward,term,_,_ = env.step(policy_model.predict(obs)[0])
        total_reward += reward
        if term==1:
            obs,_ = env.reset()
    model_reward = total_reward
    total_reward = 0
    for i in range(N):
        obs,reward,term,_,_ = env.step(data_optim(env.returns))
        total_reward += reward
        if term==1:
            obs,_ = env.reset()
    benchmark_reward = total_reward
    total_reward = 0
    return model_reward,benchmark_reward

policy_tester(InvestmentGame0,model0,simulated_returns=simulated_returns)


(737.7586317272794, 750.7688671394142)

## Reinforcement learning with changing markets
Instead of feeding in a single set of historical returns to our environment, we can instead simulate a new set of historical market returns each time we reset our environment.  In this way, we are now asking our policy model to look at an arbitrary historical dataset and find the optimal policy.  This is much more interesting - although it is still very much something our traditional approach would handle.

In [ ]:
class InvestmentGame1(gym.Env):
    def __init__(self,risk_free_rate=0):
        super(InvestmentGame1,self).__init__()
        self.action_space = spaces.Box(
            low=0,high=1,shape=(5,),dtype=np.float32)
        self.observation_space=spaces.Box(
            low=-1, high=1, shape=(100,5),dtype=np.float32)
        self.reward_range=(-10,10)

        self.mu,self.Sig = generate_market_params(5,100)
        self.returns = generate_market_returns(self.mu,self.Sig,100)

        self.risk_free_rate = risk_free_rate

    def step(self,action):
        self.current_action = action
        if action.sum()==0:
            self.weights = (action+1)/5
        else:
            self.weights = action/action.sum()

        port_returns = self.returns@self.weights
        sharpe = (port_returns.mean()-self.risk_free_rate)/port_returns.std()
        reward = sharpe.clip(-10,10)
        terminated = 1
        obs =self._get_obs()
        info = {"data":1}
        return obs,reward,terminated,0,info
    def reset(self,seed=None):
        if not seed is None:
            np.random.seed(seed)

        self.mu,self.Sig = generate_market_params(5,100)
        self.returns = generate_market_returns(self.mu,self.Sig,100)

        obs = self._get_obs()
        return obs, {'data':1}
    def _get_obs(self):
        obs = self.returns
        return obs

In [ ]:
env = DummyVecEnv([lambda: InvestmentGame1()])
policy_kwargs = dict(activation_fn=th.nn.ReLU,
                    net_arch=[dict(pi=[128,64,32],vf=[128,64,32])])
model1 = PPO(MlpPolicy, env, learning_rate=0.02, batch_size=1000, gamma=1, n_steps=10000, policy_kwargs=policy_kwargs, verbose=1)
obs = env.reset()
model1.learn(total_timesteps=300000)

Using cpu device


C:\Users\tlhee\AppData\Local\Continuum\anaconda3\envs\port_mgmt\Lib\site-packages\stable_baselines3\common\policies.py:484: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


------------------------------
| time/              |       |
|    fps             | 843   |
|    iterations      | 1     |
|    time_elapsed    | 11    |
|    total_timesteps | 10000 |
------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 780        |
|    iterations           | 2          |
|    time_elapsed         | 25         |
|    total_timesteps      | 20000      |
| train/                  |            |
|    approx_kl            | 0.33151644 |
|    clip_fraction        | 0.641      |
|    clip_range           | 0.2        |
|    entropy_loss         | -7         |
|    explained_variance   | 0.00269    |
|    learning_rate        | 0.02       |
|    loss                 | -0.0977    |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.108     |
|    std                  | 0.965      |
|    value_loss           | 0.143      |
----------------------------------------
----

---------------------------------------
| time/                   |           |
|    fps                  | 689       |
|    iterations           | 13        |
|    time_elapsed         | 188       |
|    total_timesteps      | 130000    |
| train/                  |           |
|    approx_kl            | 1.1037052 |
|    clip_fraction        | 0.801     |
|    clip_range           | 0.2       |
|    entropy_loss         | -3.62     |
|    explained_variance   | 0.647     |
|    learning_rate        | 0.02      |
|    loss                 | -0.115    |
|    n_updates            | 120       |
|    policy_gradient_loss | -0.0435   |
|    std                  | 0.495     |
|    value_loss           | 0.0252    |
---------------------------------------
---------------------------------------
| time/                   |           |
|    fps                  | 694       |
|    iterations           | 14        |
|    time_elapsed         | 201       |
|    total_timesteps      | 140000    |


---------------------------------------
| time/                   |           |
|    fps                  | 717       |
|    iterations           | 24        |
|    time_elapsed         | 334       |
|    total_timesteps      | 240000    |
| train/                  |           |
|    approx_kl            | 1.6053839 |
|    clip_fraction        | 0.835     |
|    clip_range           | 0.2       |
|    entropy_loss         | -1.56     |
|    explained_variance   | 0.697     |
|    learning_rate        | 0.02      |
|    loss                 | -0.0588   |
|    n_updates            | 230       |
|    policy_gradient_loss | 0.0177    |
|    std                  | 0.323     |
|    value_loss           | 0.0231    |
---------------------------------------
---------------------------------------
| time/                   |           |
|    fps                  | 719       |
|    iterations           | 25        |
|    time_elapsed         | 347       |
|    total_timesteps      | 250000    |


### The model is at least better than an even split
It is not much better at matching the traditional model than simply providing an even split across classes, but it is better than our previous policy model!  Maybe the difference between the policy model and the traditional model are the result of the reinforcement learning model finding some secret solution?

In [ ]:
model_sq_dif,bchmk_sq_dif = test_model_variance(model1)
np.mean(model_sq_dif),np.mean(bchmk_sq_dif)

(0.32153623244537766, 0.39182820000000007)

### Looking at the rewards again
The policy model is inferior to just using the traditional model.  This could be remedied by additional training, using a different model architecture, adjusting parameters in the PPO alogrithm, etc.  However, it does beg the question, why bother with all of this machinery.

In [ ]:
policy_tester(InvestmentGame1,model1)

(605.3167785297682, 748.8410838916747)

## A (more) realistic scenario
Unlike our previous example, we can set up a more complex market environment.  In this case, we will have a hidden market state - depending on the state of the market, there will be a different underlying data generating process (i.e. different $\mu$ and $\Sigma$.

In addition, there is now a cost associated with changing an asset allocation.  This requires us to set this environment up to persist over a period of time (30 periods in this example).

We could add additional complexities to the environment as well, such as assets with contingent or deferred payoffs, adversion to risk, taxes, additional data sources, etc.  This is where ML methods can really add value.  

In [ ]:
class InvestmentGame2(gym.Env):
    """"""
    def __init__(self,risk_free_rate=0, N=100):
        """"""
        super(InvestmentGame2,self).__init__()
        self.action_space = spaces.Box(
            low=0,high=1,shape=(5,),dtype=np.float32)
        self.observation_space=spaces.Box(
            low=-1, high=1, shape=(5+5*100+1,),dtype=np.float32)
        self.reward_range=(-3,3)
        self.N = N

        self.risk_free_rate = risk_free_rate

        self.time = 0
        self.state_trans_matrix = np.array([[.8,.2]
                                           ,[.2,.8]])

    def step(self,action):
        """"""
        self.current_action = action
        if action.sum()==0:
            self.new_weights = (action+1)/5
        else:
            self.new_weights = action/action.sum()

        self.returns = np.vstack((self.returns[1:,:],self._generate_returns()))

        port_returns = self.returns[-1,:]@self.new_weights.reshape(-1,)
        change_in_port = np.abs(self.new_weights - self.weights).sum()
        cost = 0.01*change_in_port
        self.weights = self.new_weights
        reward = port_returns - cost
        self.time += 1
        sharpe = (port_returns.mean()-self.risk_free_rate)/port_returns.std()

        if self.time == 30:
            terminated = 1
        else:
            terminated = 0

        info = {"data":1}
        return obs,reward,terminated,0,info

    def reset(self,seed=None):
        """"""
        if not seed is None:
            np.random.seed(seed)

        self.params = []
        self.params.append(generate_market_params(5,self.N))
        self.params.append(generate_market_params(5,self.N))
        self.state = np.random.choice(2)

        for i in range(self.N):
            if i == 0:
                self.returns = self._generate_returns()
            else:
                self.returns = np.vstack((self.returns,self._generate_returns()))

        self.weights = np.exp(np.random.normal(size=(5,)))
        self.weights = self.weights/self.weights.sum()

        self.time = 0

        obs = self._get_obs()
        return obs, {'data':1}

    def _get_obs(self):
        """"""
        obs = np.hstack((self.time/30,self.weights,self.returns.reshape(-1)))
        return obs

    def _generate_returns(self):
        """"""
        self.state = np.random.choice(2,p=self.state_trans_matrix[self.state,:])
        self.mu,self.Sigma = self.params[self.state]
        return generate_market_returns(self.mu,self.Sigma,1)

In [ ]:
env = DummyVecEnv([lambda: InvestmentGame2()])
policy_kwargs = dict(activation_fn=th.nn.ReLU,
                    net_arch=[dict(pi=[128,64,32],vf=[128,64,32])])
model2 = PPO(MlpPolicy, env, learning_rate=0.02, batch_size=1000, gamma=1, n_steps=10000, policy_kwargs=policy_kwargs, verbose=1)
obs = env.reset()
model2.learn(total_timesteps=300000)

Using cpu device


C:\Users\tlhee\AppData\Local\Temp\ipykernel_18252\2947046622.py:35: RuntimeWarning: divide by zero encountered in scalar divide
  sharpe = (port_returns.mean()-self.risk_free_rate)/port_returns.std()


------------------------------
| time/              |       |
|    fps             | 571   |
|    iterations      | 1     |
|    time_elapsed    | 17    |
|    total_timesteps | 10000 |
------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 521         |
|    iterations           | 2           |
|    time_elapsed         | 38          |
|    total_timesteps      | 20000       |
| train/                  |             |
|    approx_kl            | 0.048117943 |
|    clip_fraction        | 0.0591      |
|    clip_range           | 0.2         |
|    entropy_loss         | -7.08       |
|    explained_variance   | -0.00371    |
|    learning_rate        | 0.02        |
|    loss                 | -0.00339    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00336    |
|    std                  | 0.993       |
|    value_loss           | 0.00598     |
---------------------------

-----------------------------------------
| time/                   |             |
|    fps                  | 506         |
|    iterations           | 13          |
|    time_elapsed         | 256         |
|    total_timesteps      | 130000      |
| train/                  |             |
|    approx_kl            | 0.036452822 |
|    clip_fraction        | 0.0628      |
|    clip_range           | 0.2         |
|    entropy_loss         | -6.87       |
|    explained_variance   | 0.0518      |
|    learning_rate        | 0.02        |
|    loss                 | 0.00128     |
|    n_updates            | 120         |
|    policy_gradient_loss | -0.00245    |
|    std                  | 0.945       |
|    value_loss           | 0.0106      |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 504         |
|    iterations           | 14          |
|    time_elapsed         | 277   

------------------------------------------
| time/                   |              |
|    fps                  | 499          |
|    iterations           | 24           |
|    time_elapsed         | 480          |
|    total_timesteps      | 240000       |
| train/                  |              |
|    approx_kl            | 0.0065883645 |
|    clip_fraction        | 0.0679       |
|    clip_range           | 0.2          |
|    entropy_loss         | -6.45        |
|    explained_variance   | 0.0529       |
|    learning_rate        | 0.02         |
|    loss                 | 0.00221      |
|    n_updates            | 230          |
|    policy_gradient_loss | -0.0032      |
|    std                  | 0.878        |
|    value_loss           | 0.00762      |
------------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 496         |
|    iterations           | 25          |
|    time_elaps

### Get used to disappointment
In this example, the policy model we learned is still not sufficient to out-perform the traditional optimization approach.  Keep this in mind - machine learning in financial markets is difficult.  There are a few fundamental reasons for this.  First, there is limited data and it is very easy to overfit to limited historical examples.  Second, there are many market participants all looking to get an edge - if an opportunity is discovered, it is quickly exploited and will disappear.  In other words, the data are not stationary - the markets keep adapting based on the behavior of market participants (as well as to changing in policy set by governing bodies).

In [ ]:
policy_tester(InvestmentGame2,model2)

C:\Users\tlhee\AppData\Local\Temp\ipykernel_18252\2051062824.py:35: RuntimeWarning: divide by zero encountered in scalar divide
  sharpe = (port_returns.mean()-self.risk_free_rate)/port_returns.std()


(-7.837197064763865, 2.4560488765826367)